
## Make GT for reference


In [1]:
# env
import os
import cv2
import numpy as np
import time


In [15]:
# make gt inference (540p)
start = time.time()
assert os.system("python inference.py") == 0
end = time.time()
print("Time elapsed: ", end - start)

Time elapsed:  8.720263004302979


In [14]:
# compare ref
# read check.png and ref.png and compare them with np
out = cv2.imread("check.png")
ref = cv2.imread("ref.png")
np.allclose(out, ref, atol=1, equal_nan=True)

True

In [23]:
gmf_ncnn_out = cv2.imread(r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\images\out.png")

## gen pt

```
.\pnnx.exe D:/60-fps-Project/VFI/GMFSS2NCNN/flownet_288.pt inputshape=[1,3,288,480]f32,[1,3,288,480]f32 device=cpu moduleop=model.gmflow.utils.split_feature,model.gmflow.utils.merge_splits,model.gmflow.utils.convex_upsampling

(unable)
ncnnoptimize.exe D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.param D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.bin D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.opt.param D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.opt.bin 1
```

copy generated bin to gmfss model dir

In [24]:
# compare pt with ref pt
import numpy as np
output = np.load("output.npy")
ref = np.load("ref_output.npy")
np.allclose(output, ref, atol=1, equal_nan=True)


False

## Clean pnnx.py

In [ ]:
# Useless
with open("flownet_288_pnnx.py", 'r', encoding='utf-8') as r:
    content = r.read()
content = content.replace("", "")
with open("flownet_288_pnnx.py", 'w', encoding='utf-8') as w:
    w.write(content)


# Edit NCNN Param

Replace InstanceNorm
```
nn\.InstanceNorm2d(\s+)(.*?)(\s+)([\d+\s]+)
InstanceNorm$1$2$3$4 2=0
```

after:
```python
v_964 = F.upsample(input=v_963, align_corners=True, mode='bilinear', scale_factor=(2.000000,2.000000))
v_965 = (v_964 * 2)
```
all split, merge, s=8

In [53]:
# flownet
import os, shutil, re
param_path = "flownet_288.ncnn.param"
param_bak_path = "flownet_288.ncnn_bak.param"
if os.path.exists(param_bak_path):
    os.remove(param_bak_path)
shutil.copy(param_path, param_bak_path)
content = ""
is_s_8 = False
with open(param_bak_path, 'r', encoding='utf-8') as r:
    for l in r.readlines():
        if "Interp" in l:
            is_s_8 = True
        if re.search("nn.InstanceNorm2d", l, re.I):
                l = l.replace("nn.InstanceNorm2d", "InstanceNorm")
                l = f"{l.strip()} 2=0\n"
        if is_s_8:
            if re.search("split_feature|merge_splits", l, re.I):
                l = f"{l.strip()} 0=8\n"
        if re.search("convex_upsampling", l, re.I):
            l = f"{l.strip()} 0=4\n"
        content += l

with open(param_path, 'w', encoding='utf-8') as w:
    w.write(content)

# Compare with PT GT

In [56]:
# flownet
import re
import numpy as np
torch_output_mat = r"D:\60-fps-Project\VFI\GMFSS2NCNN\debug4_480.288.1.2_flow_up.txt"
ncnn_output_mat = r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\src\out\build\x64-Debug\debug2_480.288.1.2_flow_padded.txt"
with open(ncnn_output_mat, 'r', encoding='utf-8') as r:
    print(r.read()[-2000:-1000])
with open(torch_output_mat, 'r', encoding='utf-8') as r:
    print(r.read()[-2000:-1000])

9.792 9.803 9.801 9.769 9.846 9.875 9.880 9.884 9.905 9.944 9.982 10.020 9.896 9.916 9.941 9.963 10.041 10.012 10.005 10.004 9.973 9.966 9.928 9.898 9.915 9.889 9.876 9.865 9.819 9.808 9.796 9.784 9.807 9.799 9.789 9.773 9.756 9.735 9.728 9.723 9.737 9.603 9.538 9.533 9.518 9.486 9.440 9.408 9.390 9.380 9.368 9.360 9.355 9.355 9.354 9.353 9.351 9.354 9.357 9.360 9.366 9.374 9.380 9.386 9.390 9.395 9.401 9.409 9.403 9.408 9.415 9.425 9.431 9.437 9.446 9.458 9.464 9.489 9.501 9.505 9.404 9.399 9.393 9.388 9.361 9.347 9.332 9.317 9.354 9.348 9.343 9.338 9.311 9.317 9.321 9.325 9.335 9.335 9.336 9.336 9.322 9.320 9.318 9.315 9.339 9.341 9.345 9.348 9.340 9.358 9.375 9.393 9.392 9.422 9.442 9.467 9.478 9.597 9.643 9.658 9.572 9.572 9.578 9.591 9.658 9.665 9.670 9.674 9.608 9.612 9.613 9.613 9.651 9.618 9.597 9.590 9.590 9.531 9.518 9.514 9.506 9.413 9.308 9.284 9.264 9.248 9.236 9.227 9.183 9.170 9.157 9.142 9.154 9.137 9.120 9.100 9.090 9.075 9.061 9.048 9.041 9.035 9.029 9.019 9.014 9.011

In [ ]:
import re
import numpy as np
torch_output_mat = r"D:\60-fps-Project\VFI\GMFSS2NCNN\debug6_60.36.1.128_merge_splits_top.txt"
ncnn_output_mat = r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\src\out\build\x64-Debug\debug5_60.36.1.128_merge_splits_top.txt"
ncnn_output_mat = r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\src\out\build\x64-Debug\debug2_3.3.2.4_testOut.txt"

class PrintData:
    def __init__(self, r) -> None:
        shape_info = r.readline()
        shape_info = re.findall(r"shape (\d+) (\d+) (\d+) (\d+)", shape_info)[0]
        self.w, self.h, self.d, self.c = map(int, shape_info)
        data = r.readline().split(' ')
        data_ = list()
        for d in data:
            try:
                data_.append(float(d))
            except:
                print(f"failing: {d}")
        self.data = np.array(data_).astype(np.float32)
        self.data = np.pad(self.data, ((0, self.w * self.h * self.d * self.c - len(data_)),), 'constant', constant_values=(0, 0))
        self.data = self.data[:self.w * self.h * self.d * self.c].reshape(self.c, self.d, self.h, self.w)

# with open(torch_output_mat, 'r', encoding='utf-8') as r:
#     torch_output = PrintData(r)
with open(ncnn_output_mat, 'r', encoding='utf-8') as r:
    ncnn_output = PrintData(r)


In [59]:
print(torch_output.data[0, 0, 0, 30])
print(ncnn_output.data[0, 0, 0, 30])
# torch_output.data.shape

0.545
0.392


# Test Reference

In [67]:
import torch
from model.gmflow.utils import split_feature, merge_splits, convex_upsampling
# generate (2,2,1,2) tensor
feat = torch.arange(0, 72).reshape(1, 2, 6, 6)
splits = split_feature()(feat, 2)
print(splits)

merged = merge_splits()(splits, 2)
print(merged)


tensor([[[[[ 0,  1,  2],
           [ 6,  7,  8],
           [12, 13, 14]],

          [[36, 37, 38],
           [42, 43, 44],
           [48, 49, 50]]],


         [[[ 3,  4,  5],
           [ 9, 10, 11],
           [15, 16, 17]],

          [[39, 40, 41],
           [45, 46, 47],
           [51, 52, 53]]],


         [[[18, 19, 20],
           [24, 25, 26],
           [30, 31, 32]],

          [[54, 55, 56],
           [60, 61, 62],
           [66, 67, 68]]],


         [[[21, 22, 23],
           [27, 28, 29],
           [33, 34, 35]],

          [[57, 58, 59],
           [63, 64, 65],
           [69, 70, 71]]]]])
tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11],
          [12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23],
          [24, 25, 26, 27, 28, 29],
          [30, 31, 32, 33, 34, 35]],

         [[36, 37, 38, 39, 40, 41],
          [42, 43, 44, 45, 46, 47],
          [48, 49, 50, 51, 52, 53],
          [54, 55, 56, 57, 58, 59],
          [60,

In [68]:
flow = torch.arange(0, 12).reshape(1, 2, 2, 3)
up_flow = torch.arange(0, 48).reshape(1, 2, 2*2, 2*3)
print(up_flow)
up_flow = convex_upsampling(2)(flow, up_flow)
print(up_flow)



tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11],
          [12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23]],

         [[24, 25, 26, 27, 28, 29],
          [30, 31, 32, 33, 34, 35],
          [36, 37, 38, 39, 40, 41],
          [42, 43, 44, 45, 46, 47]]]])
tensor([[[[ 0,  6,  1,  7,  2,  8],
          [12, 18, 13, 19, 14, 20],
          [ 3,  9,  4, 10,  5, 11],
          [15, 21, 16, 22, 17, 23]],

         [[24, 30, 25, 31, 26, 32],
          [36, 42, 37, 43, 38, 44],
          [27, 33, 28, 34, 29, 35],
          [39, 45, 40, 46, 41, 47]]]])


In [6]:
import torch
import torchvision.models as models

net = models.resnet18(pretrained=False)
net = net.eval()

x = torch.rand(1, 3, 224, 224)

# You could try disabling checking when tracing raises error
# mod = torch.jit.trace(net, x, check_trace=False)
mod = torch.jit.trace(net, x)

mod.save("resnet18.pt")

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
